# OD Backup controller Align
At some point the rotatation arm can get out of zero and will need to be adjusted.  This helps you do that process.


To align (Servo version) follow the instruction in this notebook.

- Test alignment
- Make sure coarse alignment is ok
- Fine tune
- Test pickup and drop

## CDBurnerXP Driver Controller

In [1]:
# To allow graphs
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from od_media import ODMedia

media = ODMedia()

Found D drive all ok


## Robot Control
This imports the code.

In [3]:
import time

import od_control

robot = od_control.ODRobot(media.disk_open, media.disk_close, com_port="COM6")

Robot>import control1
>>> 


# Toolhead Calibration

In [4]:
def make_safe(try_again = True):
    if robot.z_at_top():
        print('at top')
        return True        
    elif robot.z_at_bottom():
        print('at bottom')
        robot.zero_toolhead()
        if try_again:
            return make_safe(try_again = False)         
        else:
            return False
    elif robot.z_in_middle():
        print('in middle')
        robot.zero_toolhead()
        if try_again:
            return make_safe(try_again = False)         
        else:
            return False
    else:
        print('Toolhead in error condition as not at top, bottom or middle')
        raise Exception('Toolhead in unknown position - needs debugging')
make_safe()

in middle
at top


True

In [ ]:
# Test the DVD
robot.disk_close()
robot.rotator.to_bin('od')  # Make sure rotator in correct position
r1 = robot.measure_up_down(speed_offset=7, offset_middle=50)  
# 7 offset from  seems ok large enough to get out of dead band and small enough not to go too fast
# 50 Seems a general starting position if starting from scratch

r1

In [ ]:
r2 = robot.measure_up_down(speed_offset=5, offset_middle=r1[2])  # Recalculate slower and using revised estimate
# 6 offset is max before inertia takes place and it over stops. You can see this on a move up
# where the top bends a bit as it stops
# So use 5 for production  (was 4 but improved stopping algorithm)
r2

In [ ]:
# Now need to update driver with new calibration data for toolhead driving speed
robot.calibration['z_axis_down_setting'] = r2[0][0]
robot.calibration['z_axis_up_setting'] = r2[1][0]
robot.save_calibration()

## Test how center varies with time

In [ ]:
def get_center():
    r = robot.measure_up_down(speed_offset=6)  # Use existing middle as estaimte
    return r[2]

In [ ]:
get_center()

In [ ]:
import time
from matplotlib import animation
from JSAnimation import IPython_display

num_points = 10

fig = plt.figure()
ax = plt.axes(xlim=(0, num_points), ylim=(40, 60))
line, = ax.plot([], [], lw=2)

raw = np.zeros(num_points) # scale repeats to y axis limits


def init():
    line.set_data([], [])
    return line,

def animate(i):
    x = np.linspace(0, i, i) # scale repeats to y axis limits
    raw[i] = get_center() #50 + np.cos(0.02) * np.sin(i)
    y = raw[:i] #50 + np.cos(0.02 * np.pi) * np.sin(x)
    line.set_data(x, y)
    return line,

#def animate(i):
#    # global x, y
#    x = np.linspace(0, num_points, num_points) # scale repeats to y axis limits
#    y = x * 2 #np.linspace(0, num_points, num_points) # scale repeats to y axis limits##

    # y[i] = 50 + cos(0.02 * np.pi) * sin(x[i])
    #line.set_data(x, y)
    #time.sleep(1)
    #return line,

#Animation does not run live but all the data is collected first
#animation.FuncAnimation(fig, animate, init_func=init,
#                        frames=num_points, interval=30)  # so doing it num_points times

## Now start the list of commands to align the rotator

In [ ]:
# Test the DVD
robot.disk_open()
robot.rotator.to_bin('od')
robot.move_toolhead_to(15)

## Check Coarse alignment
The toolhead should be above the CD drive. Put in a test CD to check alignment.
**If the toolhead is not close (Just beside hole) then**:
- Turn power off
- Rotate head by hand 
- Power on *Drive will be closed*
- Restart kernel and clear outputs
- Restart this notebook

## Fine tune alignement
Adjust defaults in code (TODO store in device)
- First align height so just above surface
- Then find correct rotation angle
- then save correct

In [ ]:
# Fine tune head height by manual inspection.
print(f'Current position = {robot.toolhead_z}, lower is higher')

In [ ]:
robot.move_toolhead_to(26)

In [ ]:
# Fine tune rotation number
print(f"Current position = {robot.rotator.old_pos}, lower is counter clockwise, higher clockwise")
print(f"Centre = {robot.rotator.bins['od'][0]}"
      f", offset load = {robot.rotator.bins['od'][1]}"
      f", offset unload = {robot.rotator.bins['od'][2]}")


In [ ]:
new_pos = 49.75 # Actual position regardless of offset
robot.rotator.rotate_head_anti_hysteresis(new_pos, verbose=True)

In [ ]:
#Now save calibration number
calib = robot.calibration['bins']['od']
calib[0] = new_pos - calib[2]  # Subtract out the offset
robot.save_calibration()
robot.move_toolhead_to(15)
# Test
robot.rotator.to_bin('od')

# Calibrate speed/distance of toolhead by using DVD tray
# *The rotation 
must be calibrated or it will self destruct*
Need to make sure tray is open and has test OD.  This should be loaded by hand.

In [ ]:
robot.zero_toolhead()  # Make sure toolhead out of the way


In [ ]:
# Test the DVD
robot.disk_open()
robot.rotator.to_bin('od')

Make sure disk is in drive and aligned

In [ ]:
robot.zero_toolhead()

now going to lower tool head until it hits the disc and stops (hopefully) and timeit.
Then time going up

In [ ]:
cr1 = robot.measure_up_down(time_down=100)  # long down so hits end stop
cr1

In [ ]:
robot.move_toolhead_to(45)

In [ ]:
# Calculate speed
# Although a) saw a quick retraction
# b) move_to speeds seem slow compared to up_down
# When closed measure distance between exposed track of toolhead 
track_closed = 6.6  # mm
track_open = 47.4  # mm
track_calibration = track_open - track_closed  # mm
print(f'Distance travelled from top to disc in Optical disc is {track_calibration} mm')
speed_down =  track_calibration / cr1[0][1] #mm/s
speed_up =  track_calibration / cr1[1][1] #mm/s
average_speed  = (speed_down + speed_up) / 2
print(f'Average speed = {average_speed:.2f}, down = {speed_down:.2f}, up = {speed_up:.2f} (in mm/s)')
robot.calibration['toolhead_speed_up'] = speed_up
robot.calibration['toolhead_speed_down'] = speed_down
robot.save_calibration()

## Test pickup and drop on Optical Drive
Assuming disc in OD and open

In [ ]:
# Check grip, release, park and shutdown work
robot.grip()
robot.release()
robot.park()
robot.shutdown()

In [ ]:
robot.pickup_from_bin('od')
# If this does not happen smoothly then a fine adjustment of set posisition could help

In [ ]:
robot.drop_on_bin('od')
# Disk should be centered in drive

## Repeat Input bin
Adjust defaults in code (TODO store in device)
- First align height so just above surface
- Then find correct rotation angle
- then save correct
First test alignment
Then check pickup and drop

In [ ]:
make_safe()

In [ ]:
make_safe()
robot.rotator.to_bin('in')
robot.move_toolhead_to(10)

Check is aligned if not:
- align rotation with hole
- align holder with hole

In [5]:
# Fine tune rotation number
print(f"Current position = {robot.rotator.old_pos}, lower is counter clockwise, higher clockwise")
print(f"Centre = {robot.rotator.bins['in'][0]}"
      f", offset load = {robot.rotator.bins['in'][1]}"
      f", offset unload = {robot.rotator.bins['in'][2]}")

Current position = -1, lower is counter clockwise, higher clockwise
Centre = 86, offset load = 0, offset unload = 0


In [6]:
new_pos = 88 # Actual position regardless of offset
robot.rotator.rotate_head_anti_hysteresis(new_pos, verbose=True)

Robot>control1.rotate.wait_set_position(88)
>>> 


In [7]:
robot.move_toolhead_to(25)

In [8]:
#Now save calibration number
calib = robot.calibration['bins']['in']
calib[0] = new_pos - calib[2]  # Subtract out the offset
robot.save_calibration()
robot.move_toolhead_to(15)
# Test
robot.rotator.to_bin('in')

**Make SAFE and Save calibration** before testing

In [9]:
make_safe()

in middle
at top


True

In [10]:
robot.pickup_from_bin('in')

Robot>control1.th.release()
>>> 
Robot>control1.z_axis.nudge(53.62222356279805,run_time=60)
Shutdown Z Axis nudge
>>> 
Robot>control1.th.grip()
>>> 


In [11]:
robot.drop_on_bin('in')

Robot>control1.z_axis.nudge(53.62222356279805,run_time=10)
Shutdown Z Axis nudge
>>> 
Robot>control1.th.release()
>>> 


## Test pickup and drop on Out Bin
First test alignment
Then check pickup and drop.  Make sure have disck in out bin

In [12]:
make_safe()
robot.rotator.to_bin('out')
robot.move_toolhead_to(10)

at top


In [ ]:
media.disk_close()

In [ ]:
media.disk_open()

In [13]:
# Get disc out of tray to out bin
robot.pickup_from_bin('od')
robot.drop_on_bin('out')

Robot>control1.th.release()
>>> 
Robot>control1.z_axis.nudge(53.62222356279805,run_time=70)
Shutdown Z Axis nudge
>>> 
Robot>control1.th.grip()
>>> 
Robot>control1.z_axis.nudge(53.62222356279805,run_time=10)
Shutdown Z Axis nudge
>>> 
Robot>control1.th.release()
>>> 


In [ ]:
robot.pickup_from_bin('out')

In [ ]:
robot.drop_on_bin('out')

In [ ]:
robot.drop_on_out_bin()

## Testing waste bin
Make sure you have a test disk on the source.
Remember with cannot pickup from waste

In [ ]:
robot.pickup_from_bin('od')

In [ ]:
robot.drop_on_bin('waste')

## Test unloading from empty optical drive bin

In [ ]:
robot.unload()

In [ ]:
robot.load()

In [ ]:
robot.disk_close()

# Blank unload  test
The result was *Not safe*.  The toolhead went through the hole in the DVD drive and kept on going.  
I suppose you can test by using the DVD drive to check that the drive has a disk loaded before unloading.

In [ ]:
robot.rotator.to_bin('od')

In [ ]:
robot.robot(f'control1.z_axis.pwm_c()', verbose=True)

In [ ]:
# check next disk blank
robot.pickup_from_bin('in')
robot.drop_on_bin('od')

In [ ]:
# put blank disk back
media.disk_open()
robot.pickup_from_bin('od')
robot.drop_on_bin('in')